In [1]:
pip install pyspark==3.2

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyspark as ps

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google'

In [ ]:
file="/content/gdrive/MyDrive/TweetSentimentAnalysisCategoryAndVisualization/PredictedDataset/part-00000-1a6b304d-5422-4b8a-be02-064539e99158-c000.csv"
file1="/content/gdrive/MyDrive/TweetSentimentAnalysisCategoryAndVisualization/PredictedDataset/part-00001-1a6b304d-5422-4b8a-be02-064539e99158-c000.csv"
file2="/content/gdrive/MyDrive/TweetSentimentAnalysisCategoryAndVisualization/PredictedDataset/part-00002-1a6b304d-5422-4b8a-be02-064539e99158-c000.csv"
file3="/content/gdrive/MyDrive/TweetSentimentAnalysisCategoryAndVisualization/PredictedDataset/part-00003-1a6b304d-5422-4b8a-be02-064539e99158-c000.csv"
file4="/content/gdrive/MyDrive/TweetSentimentAnalysisCategoryAndVisualization/PredictedDataset/part-00004-1a6b304d-5422-4b8a-be02-064539e99158-c000.csv"
file5="/content/gdrive/MyDrive/TweetSentimentAnalysisCategoryAndVisualization/PredictedDataset/part-00005-1a6b304d-5422-4b8a-be02-064539e99158-c000.csv"

In [ ]:
#sc.stop()
conf = ps.SparkConf().setAll([('spark.executor.memory', '16g'), ('spark.driver.memory', '16g')])
sc = ps.SparkContext(conf=conf)

In [ ]:
import functools
  
def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

In [ ]:
from pyspark.sql import SQLContext

sql_context = SQLContext(sc)
TwitterDataset = sql_context.read.format('com.databricks.spark.csv').option('header','false').option("delimiter", ",").load(file)
TwitterDataset1 = sql_context.read.format('com.databricks.spark.csv').option('header','false').option("delimiter", ",").load(file1)
TwitterDataset2 = sql_context.read.format('com.databricks.spark.csv').option('header','false').option("delimiter", ",").load(file2)
TwitterDataset3 = sql_context.read.format('com.databricks.spark.csv').option('header','false').option("delimiter", ",").load(file3)
TwitterDataset4 = sql_context.read.format('com.databricks.spark.csv').option('header','false').option("delimiter", ",").load(file4)
TwitterDataset5 = sql_context.read.format('com.databricks.spark.csv').option('header','false').option("delimiter", ",").load(file5)

In [ ]:

TwitterDataset = unionAll([TwitterDataset, TwitterDataset2])

In [ ]:
TwitterDataset = unionAll([TwitterDataset, TwitterDataset1])

In [ ]:
TwitterDataset = unionAll([TwitterDataset, TwitterDataset3])

In [ ]:
TwitterDataset = unionAll([TwitterDataset, TwitterDataset4])

In [ ]:
TwitterDataset = unionAll([TwitterDataset, TwitterDataset5])

In [ ]:
print(TwitterDataset.count())

In [ ]:
TwitterDataset.show()

In [ ]:
query_1_source = TwitterDataset.groupBy("_c2").count()


In [ ]:
query_1_source.show()

In [ ]:
TwitterDataset.createOrReplaceTempView("tweets")

In [ ]:
query_1_source = sql_context.sql("SELECT cast(count(*) as int) as count , _c2 as categories FROM tweets WHERE _c2 ='political' or _c2='tech' or _c2='business' GROUP BY _c2 ORDER BY count desc")

In [ ]:
query_1_source = query_1_source.toPandas()


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (10, 8))
import seaborn as sns
import pandas
sns.barplot( x="categories", y="count", data=query_1_source)
plt.show()

In [ ]:
from pyspark.sql.functions import col
#sentimentDataSet = TwitterDataset.select(col("_c1").alias("Sentiment"))
query_1_source = sql_context.sql("SELECT * FROM tweets WHERE _c2 ='political' or _c2='tech' or _c2='business' ")

In [ ]:
from pyspark.sql.functions import when
sentimentDataSet=query_1_source.withColumn("_c1", when(col("_c1") == 0, "Negitive").otherwise("Positive"))

In [ ]:
#airline_tweets.airline_sentiment.value_counts().plot(kind='pie', autopct='%1.0f%%', colors=["red", "yellow", "green"])

query_3_source = sentimentDataSet.toPandas()
print(query_3_source.columns)
query_3_source._c1.value_counts().plot(kind='pie', autopct='%1.0f%%', colors=["gray", "red"])

In [ ]:
sentimentcategoryDataSet=sql_context.sql("SELECT * FROM tweets WHERE _c2 ='political' or _c2='tech' or _c2='business' ")

In [ ]:
sentimentcategoryDataSet

In [ ]:
sentimentcategoryDataSet = sentimentcategoryDataSet.select(col("_c1").alias("Sentiment"),col("_c2"),col("_c0"))
sentimentcategoryDataSet=sentimentcategoryDataSet.withColumn("Sentiment", when(col("Sentiment") == 0, "Negitive").otherwise("Positive"))

In [ ]:
sentimentcategoryDataSet.show()

In [ ]:
dataSet=sentimentcategoryDataSet.toPandas()

In [ ]:

sentimentplot = dataSet.groupby(['_c2','Sentiment']).Sentiment.count().unstack()

sentimentplot.plot(kind='bar')

In [ ]:
query_2_2_source=sentimentcategoryDataSet
print(query_2_2_source.count())
query_2_2_list = query_2_2_source.select('_c0').collect()
hashtags=" "   
for item in query_2_2_list:
    hashtags = hashtags + str(item._c0).strip()
from wordcloud import WordCloud    
wordcloud = WordCloud().generate(hashtags)

plt.figure( figsize = (10, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
query_2_2_source=sentimentcategoryDataSet.filter(sentimentcategoryDataSet["_c2"] == 'tech')
print(query_2_2_source.count())
query_2_2_list = query_2_2_source.select('_c0').collect()
hashtags=" "   
for item in query_2_2_list:
    hashtags = hashtags +str(item._c0).strip()
from wordcloud import WordCloud    
wordcloud = WordCloud().generate(hashtags)

plt.figure( figsize = (10, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
query_2_2_source=sentimentcategoryDataSet.filter(sentimentcategoryDataSet["_c2"] == 'political')
print(query_2_2_source.count())
query_2_2_list = query_2_2_source.select('_c0').collect()
hashtags=" "   
for item in query_2_2_list:
    hashtags = hashtags + str(item._c0).strip()
from wordcloud import WordCloud    
wordcloud = WordCloud().generate(hashtags)

plt.figure( figsize = (10, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#bussiness
query_2_1_source=sentimentcategoryDataSet.filter(sentimentcategoryDataSet["_c2"] == 'business')
print(query_2_1_source.count())
query_2_1_list = query_2_1_source.select('_c0').collect()
hashtags=" "   
for item in query_2_1_list:
    hashtags = hashtags + str(item._c0).strip()
from wordcloud import WordCloud    
wordcloud = WordCloud().generate(hashtags)

plt.figure( figsize = (10, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
query_twitter_Tech = sql_context.sql("SELECT  *  FROM tweets WHERE _c0 Like '%twitter%' and _c1='elonmusk' ")

In [ ]:
query_twitter_Tech.show(5,truncate=False)

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.functions import when
query_twitter_Tech = query_twitter_Tech.select(col("_c2").alias("Sentiment"),col("_c1"),col("_c0"))

In [ ]:
query_twitter_Tech=query_twitter_Tech.withColumn("Sentiment", when(col("Sentiment") == 0, "Negitive").otherwise("Positive"))

In [ ]:
twittertech=query_twitter_Tech.count()
query = query_twitter_Tech.toPandas()
print(query.columns)
query.Sentiment.value_counts().plot(kind='pie', autopct='%2.0f%%', colors=[ "red","green"])

In [ ]:
twittertech=query_twitter_Tech.count()

In [ ]:

query_amp_Tech = sql_context.sql("SELECT  *  FROM tweets WHERE _c0 Like '%cloud%' and _c1='tech' ")
query_amp_Tech = query_amp_Tech.select(col("_c2").alias("Sentiment"),col("_c1"),col("_c0"))
query_amp_Tech=query_amp_Tech.withColumn("Sentiment", when(col("Sentiment") == 0, "Negitive").otherwise("Positive"))
querytech=query_amp_Tech.count()
query = query_amp_Tech.toPandas()
print(query.columns)
query.Sentiment.value_counts().plot(kind='pie', autopct='%1.0f%%', colors=[ "red","green"])

In [ ]:
query_one_Tech = sql_context.sql("SELECT  *  FROM tweets WHERE _c0 Like '%one%' and _c1='tech' ")
query_one_Tech = query_one_Tech.select(col("_c2").alias("Sentiment"),col("_c1"),col("_c0"))
query_one_Tech=query_one_Tech.withColumn("Sentiment", when(col("Sentiment") == 0, "Negitive").otherwise("Positive"))
onetech=query_one_Tech.count()
query = query_one_Tech.toPandas()
print(query.columns)
query.Sentiment.value_counts().plot(kind='pie', autopct='%1.0f%%', colors=[ "red","green"])

In [ ]:
query_one_Tech = sql_context.sql("SELECT  *  FROM tweets WHERE _c0 Like '%new%' and _c1='tech' ")
query_one_Tech = query_one_Tech.select(col("_c2").alias("Sentiment"),col("_c1"),col("_c0"))
query_one_Tech=query_one_Tech.withColumn("Sentiment", when(col("Sentiment") == 0, "Negitive").otherwise("Positive"))
newtech=query_one_Tech.count()
query = query_one_Tech.toPandas()
print(query.columns)
query.Sentiment.value_counts().plot(kind='pie', autopct='%1.0f%%', colors=[ "red","green"])

In [ ]:
query_one_Tech = sql_context.sql("SELECT  *  FROM tweets WHERE _c0 Like '%make%' and _c1='tech' ")
query_one_Tech = query_one_Tech.select(col("_c2").alias("Sentiment"),col("_c1"),col("_c0"))
query_one_Tech=query_one_Tech.withColumn("Sentiment", when(col("Sentiment") == 0, "Negitive").otherwise("Positive"))
maketech=query_one_Tech.count()
query = query_one_Tech.toPandas()

print(query.columns)
query.Sentiment.value_counts().plot(kind='pie', autopct='%1.0f%%', colors=[ "red","green"])

In [ ]:
print(twittertech)
print(maketech)
print(newtech)
print(onetech)
print(querytech)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
x = ["one","new","amp","twitter","make"]
y = [3206,1086,1002,704,696];
ax.bar(x,y)
plt.show()